# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 14.0 MB/s eta 0:00:00
  Attempting uninstall: autoawq
    Found existing installation: autoawq 0.1.7
    Uninstalling autoawq-0.1.7:
      Successfully uninstalled autoawq-0.1.7


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "01-ai/Yi-34B-Chat-4bits"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype='auto',
    device_map="auto"
).eval()

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    messages = [{"role": "user", "content": input_text}]
    input_ids = tokenizer.apply_chat_template(
        conversation=messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt'
    )
    return input_ids

def generate_text(input_ids):
    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    output_ids = model.generate(
        input_ids.to('cuda'),
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.8,
    )

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)
    return response

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            input_ids = format_prompt(prompt)
            generated_text = generate_text(input_ids)
            print(generated_text)
            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

1. 目標設定と分割：大きな目標を小さな目標に分割することで、達成感を持てる。
2. ストレッチ目標：困難度が高い目標を設定することで、より大きな成果を目指す。
3. 習慣化：日々の習慣によって、仕事に対する集中力と熱意を高める。
4. モチベーションの維持：毎日のモチベーションを維持するために、自分に対する小さな成功を経験すること。
5. 学び続ける：新しいスキルや知識を学び、仕事に新鮮感を与える。
**海辺の友達**

クマは町の中心部から車で1時間ほど離れた、静かな海辺の町に暮らしていた。彼は独り身で、仕事もあまり忙しくなく、日々の生活は平和で単調だった。クマは孤独を感じることは稀ではあったが、不思議にも、海辺の生活は何か不満を孕んでいた。

ある日、クマは習慣として朝早く海辺を散歩しに出かけた。空気は清々しく、波の音が心にひろがった。その時、彼は海に浮かんでいる何かに気がついた。それはアザラシだった。

クマは一歩一歩とアザラシのそばに近づき、その大きな目とじっくりと向き合った。何か言葉では表現できないようなつながりを感じた。アザラシはクマの手を嗅ぎ、その優しい目で彼を見つめた。そして、彼女は海に潜り込んだ。

その後の日々、クマはアザラシと出会うたびに、彼女との間に生じたその特別な繋がりを感じた。彼は海辺で彼女を待ち、彼女はいつも守るようにしていた。時間が経つにつれ、クマはアザラシに対する感情が深まった。彼女は彼の心の空っぽい部分を埋めてくれた。

ある日、クマはアザラシに話しかけた。彼女はいつも聴いてくれるし、彼の話しかけるだけでも幸福だった。彼女は彼の声を愛し、彼の気持ちを理解していた。そして、ふとしたことで、クマはアザラシに「友達」と呼ぶようになった。

しかし、季節が変わり、天候が悪くなり、クマはアザラシの姿を見ることができなくなった。彼は心配したが、何かしらの理由で彼女が出てこなかった。日々の生活は、アザラシがいなくてはならないというノイズで満たされた。

そして、一つの日、クマは突然、彼女の家に帰るべきだということを知った。彼は不安だった。彼女はいつも彼にそばを離れなかったのに、今度は彼女がいなくなるというのか。

クマは海辺を歩き、アザラシの姿を探した。彼女はいなかった。彼は波の音を聞き、空を見上げ、何かを感じた。そして、彼は彼女の姿を見た。